In [1]:
import numpy as np
import matplotlib.pylab as plt
from bullet_utils.env import BulletEnvWithGround
from robot_properties_solo.solo8wrapper import Solo8Robot, Solo8Config
import mim_control_cpp
from dynamic_graph_head import ThreadHead, SimHead, SimVicon, HoldPDController
import pinocchio as pin

In [2]:
class FootSliderController:
    def __init__(self, head, Kp, Kd, with_sliders=False):
        self.head = head
        self.Kp = Kp
        self.Kd = Kd

        self.slider_scale = np.pi
        self.with_sliders = with_sliders

        self.joint_positions = head.get_sensor('joint_positions')
        self.joint_velocities = head.get_sensor('joint_velocities')

        if with_sliders:
            self.slider_positions = head.get_sensor('slider_positions')


#         variables for linspace run
#         self.L = None
#         self.i_L = 0
#         self.go = 0
#         self.num = 700

    def warmup(self, thread_head):
        self.zero_pos = self.joint_positions.copy()

        if self.with_sliders:
            self.slider_zero_pos = self.map_sliders(self.slider_positions)

    def go_zero(self):
        # TODO: Make this an interpolation.
        self.zero_pos = np.zeros_like(self.zero_pos)

        if self.with_sliders:
            self.slider_zero_pos = self.map_sliders(self.slider_positions)

    def map_sliders(self, sliders):
        sliders_out = np.zeros_like(self.joint_positions)
        if self.joint_positions.shape[0] == 8:
            slider_A = sliders[0]
            slider_B = sliders[1]
            for i in range(4):
                sliders_out[2 * i + 0] = slider_A
                sliders_out[2 * i + 1] = 2. * (1. - slider_B)

                if i >= 2:
                    sliders_out[2 * i + 0] *= -1
                    sliders_out[2 * i + 1] *= -1

        return sliders_out

    def run(self, thread_head):
        def get_vicon(name1, name2=None):
            if name2 is None:
                name2 = name1
            pos, vel = thread_head.vicon.get_state(name1 + '/' + name2)
            return np.hstack([pos, vel])

        self.vicon_solo = get_vicon('solo8v2')
#         self.vicon_leg_fr = get_vicon('solo8_fr', 'hopper_foot')
#         self.vicon_leg_hl = get_vicon('solo8_hl', 'hopper_foot')
#         self.vicon_leg_hr = get_vicon('solo8_hr', 'hopper_foot')

        if self.with_sliders:
            self.des_position = self.slider_scale * (
                self.map_sliders(self.slider_positions) - self.slider_zero_pos
                ) + self.zero_pos 
        else:
            self.des_position = self.zero_pos

#         linspace
#         if self.go == 0:
#             self.go = 1
#             target = self.joint_positions.copy()
#             target[5] += 0.5
#             self.L = np.linspace(self.joint_positions, target, num=self.num)
#         elif self.go == 1 and self.i_L >= self.L.shape[0]:
#             self.go = 2
#             self.i_L = 0
#             target = self.joint_positions.copy()
#             target[5] -= 0.5
#             self.L = np.linspace(self.joint_positions, target, num=self.num)
#         elif self.go == 2 and self.i_L >= self.L.shape[0]:
#             self.L[self.L.shape[0]]  # switch to safety controller
        
#         self.des_position = self.L[self.i_L]
#         self.i_L += 

        self.tau = self.Kp * (self.des_position - self.joint_positions) - self.Kd * self.joint_velocities

#         self.tau[:4] = np.zeros(4)
#         self.tau[6:] = np.zeros(2)

        thread_head.head.set_control('ctrl_joint_torques', self.tau)


class CentroidalController:
    def __init__(self, head, vicon_name, mu, kp, kd, kc, dc, kb, db):
        self.set_k(kp, kd)
        self.robot = Solo8Config.buildRobotWrapper()
        self.vicon_name = vicon_name

        self.x_com = [0.0, 0.0, 0.20]
        self.xd_com = [0.0, 0.0, 0.0]

#         self.x_des = np.array([ 
#              0.2, 0.142, 0.015,  0.2, -0.142,  0.015,
#             -0.2, 0.142, 0.015, -0.2, -0.142,  0.015
#         ])
        self.x_des = np.array([ 
            0.142, 0.015, -0.142,  0.015,
            0.142, 0.015, -0.142,  0.015
        ])
        
        self.xd_des = np.array(4*[0., 0.])

        self.x_ori = [0., 0., 0., 1.]
        self.x_angvel = [0., 0., 0.]
        self.cnt_array = 4*[1,]
        
        self.w_com = np.zeros(6)
        
        q_init = np.zeros(15)
        q_init[7] = 1
        self.centrl_pd_ctrl = mim_control_cpp.CentroidalPDController()
        self.centrl_pd_ctrl.initialize(2.5, np.diag(self.robot.mass(q_init)[3:6, 3:6]))

        self.force_qp = mim_control_cpp.CentroidalForceQPController()
        self.force_qp.initialize(4, mu, np.array([5e5, 5e5, 5e5, 1e6, 1e6, 1e6]))
                
        root_name = 'universe'
        endeff_names = ['FL_ANKLE', 'FR_ANKLE', 'HL_ANKLE', 'HR_ANKLE']
        self.imp_ctrls = [mim_control_cpp.ImpedanceController() for eff_name in endeff_names]
        for i, c in enumerate(self.imp_ctrls):
            c.initialize(self.robot.model, root_name, endeff_names[i])
        
        self.kc = np.array(kc)
        self.dc = np.array(dc)
        self.kb = np.array(kb)
        self.db = np.array(db)
                
        self.joint_positions = head.get_sensor('joint_positions')
        self.joint_velocities = head.get_sensor('joint_velocities')
        self.slider_positions = head.get_sensor('slider_positions')
        self.imu_gyroscope = head.get_sensor('imu_gyroscope')

    def set_k(self, kp, kd):
        self.kp = 4 * [kp, kp, kp, 0, 0, 0]
        self.kd = 4 * [kd, kd, kd, 0, 0, 0]

    def warmup(self, thread):
        thread.vicon.bias_position(self.vicon_name)
        self.zero_sliders = self.slider_positions.copy()

    def get_base(self, thread):
        base_pos, base_vel = thread.vicon.get_state(self.vicon_name)
        base_vel[3:] = self.imu_gyroscope
        return base_pos, base_vel
    
    def run(self, thread):
        base_pos, base_vel = self.get_base(thread)

        self.q = np.hstack([base_pos, self.joint_positions])
        self.dq = np.hstack([base_vel, self.joint_velocities])

        self.w_com[:] = 0
        
        self.centrl_pd_ctrl.run(
            self.kc, self.dc, self.kb, self.db,
            self.q[:3], self.x_com, self.dq[:3], self.xd_com,
            self.q[3:7], self.x_ori, self.dq[3:6], self.x_angvel
        )
        
        self.w_com[2] = 9.81 * Solo12Config.mass
        self.w_com += self.centrl_pd_ctrl.get_wrench()
        
        if hasattr(self, 'update_w_com'):
            self.update_w_com(thread)
        
        # distrubuting forces to the active end effectors
        pin_robot = self.robot
        pin_robot.framesForwardKinematics(self.q)
        com = self.com = pin_robot.com(self.q)
        rel_eff = np.array([
            pin_robot.data.oMf[i].translation - com for i in Solo12Config.end_eff_ids
        ]).reshape(-1)
                
        ext_cnt_array = [1., 1., 1., 1.]
        self.force_qp.run(self.w_com, rel_eff, ext_cnt_array)
        self.F = self.force_qp.get_forces()
        
        if hasattr(self, 'update_F'):
            self.update_F(thread)
        
        # passing forces to the impedance controller
        self.tau = np.zeros(18)
        for i, c in enumerate(self.imp_ctrls):
            c.run(self.q, self.dq,
                 np.array(self.kp[6*i:6*(i+1)]),
                 np.array(self.kd[6*i:6*(i+1)]),
                 1.,
                 pin.SE3(np.eye(3), np.array(self.x_des[3*i:3*(i+1)])),
                 pin.Motion(self.xd_des[3*i:3*(i+1)], np.zeros(3)),
                 pin.Force(self.F[3*i:3*(i+1)], np.zeros(3))
             )

            self.tau += c.get_torques()
                
        head.set_control('ctrl_joint_torques', self.tau[6:])

In [3]:
bullet_env = BulletEnvWithGround()
robot = Solo8Robot()
bullet_env.add_robot(robot)

head = SimHead(robot, vicon_name='solo8v2')
thread_head = ThreadHead(
    0.001,  # dt
    HoldPDController(head, 3, 0.05, True),  # safety controller
    head,  # head object
    [  # vicon stuff
        ('vicon', SimVicon(['solo8v2/solo8v2']))
    ],  
    bullet_env
)

Hello world from websocket thread. <ThreadHead(Thread-4, initial)>


In [4]:
# centroidal_controller = CentroidalController(head, 'solo8v2/solo8v2', 0.2, 50., 0.7,
#     [100., 100.], [15., 15.], [25., 25.], [22.5, 22.5]
# )

foot_slider_controller = FootSliderController(head, 3, 0.05, with_sliders=True)

q0 = np.array(Solo8Config.initial_configuration)
q0[0] = 0.
q0[2] -= 0.01

dq0 = np.array(Solo8Config.initial_velocity)
thread_head.head.reset_state(q0, dq0)

# thread_head.switch_controllers(centroidal_controller)
thread_head.switch_controllers(foot_slider_controller)

In [5]:
thread_head.start_streaming()
thread_head.start_logging()

thread_head.sim_run(1*1000)

thread_head.stop_streaming()
thread_head.stop_logging()

# Plot timing information.
# thread_head.plot_timing()

140549492994000
  Not logging 'head' as field type '<class 'dynamic_graph_head.sim_head.SimHead'>' is unsupported
  Not logging 'with_sliders' as field type '<class 'bool'>' is unsupported
!!! ThreadHead: Start streaming data.
!!! ThreadHead: Start logging to file "2021-07-09_17-09-26.mds" for 30.00 seconds.
!!! ThreadHead: Stop streaming data.
!!! ThreadHead: Stop logging to file "2021-07-09_17-09-26.mds".


In [6]:
print(id(head))

140549673387232


In [7]:
print(head.__dict__.keys())

dict_keys(['_robot', '_vicon_name', '_joint_index', 'nj', '_sensor_joint_positions', '_sensor_joint_velocities', 'with_sliders', '_sensor_slider_positions', '_sensor_imu_gyroscope', '_sensor__vicon_base_position', '_sensor__vicon_base_velocity', '_control_ctrl_joint_torques', '_noise_data_std', '_fill_history_control', '_ti', '_control_delay_dt', '_history_control', '_fill_history_measurement', '_measurement_delay_dt', '_history_measurements'])


In [8]:
print(head.get_sensor('imu_gyroscope'))
print(head.get_sensor('_vicon_base_position'))

[-9.61911029e-06  2.26049250e-06 -1.90590150e-04]
[-2.11584382e-02 -4.09606611e-03  3.36892456e-01 -3.03500165e-05
  2.17904962e-04  2.53127385e-02  9.99679565e-01]


In [9]:
for i in head.__dict__.items():
    print(i[0])
    print(i[1])

_robot
_vicon_name
solo8v2
_joint_index
None
nj
8
_sensor_joint_positions
[-0.02552903 -0.01623011  0.02408209  0.01460937 -0.00873543 -0.00507375
 -0.0025534  -0.00111779]
_sensor_joint_velocities
[-2.38603421e-04  3.41753607e-04 -5.93804502e-04  3.04306374e-05
  2.35206030e-04 -5.00436627e-05 -4.37487371e-04 -4.68769889e-04]
with_sliders
True
_sensor_slider_positions
[0. 0. 0. 0.]
_sensor_imu_gyroscope
[-9.61911029e-06  2.26049250e-06 -1.90590150e-04]
_sensor__vicon_base_position
[-2.11584382e-02 -4.09606611e-03  3.36892456e-01 -3.03500165e-05
  2.17904962e-04  2.53127385e-02  9.99679565e-01]
_sensor__vicon_base_velocity
[-3.24192718e-05 -2.32861151e-05  1.65814352e-06 -9.61911029e-06
  2.26049250e-06 -1.90590150e-04]
_control_ctrl_joint_torques
[ 0.07659903  0.04867324 -0.07221659 -0.04382963  0.02619454  0.01522374
  0.00768206  0.00337679]
_noise_data_std
{'joint_positions': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'base_velocity': array([0., 0., 0., 0., 0., 0., 0., 0.]), 'imu_gyr